# Homework 3
---

*imports*

In [28]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.preprocessing import scale

%matplotlib inline


## Problem 1:

### Diagnosis of Fetal Cardio-Vascular Disease from Cardiotocography using ANN in SciKitLearn

---

#### a.
  * In this problem, I selected a new medical dataset from UCI ML repository. The dataset includes 2126 fetal cardiotocograms (CTGs) collected in the Biomedical Engineering Institute in Porto, Portugal.
  
  The dataset includes **21 feature elements**, and the output (NSP) has **3 classes**:
  
  `1: Normal; 2: Suspect; 3: Pathologic`.
  
  For more info, you can visit: https://archive.ics.uci.edu/ml/datasets/Cardiotocography You can either download the original raw data from the above link OR download the clean version of the data as a csv file from CSNS in the HW package (“CTG_clean.csv”).
  
  The goal is to build an Artificial Neural Network that can diagnose (classify) the fetal health status. The last column (NSP) in the dataset is the label: NSP - fetal state class code (`1=normal; 2=suspect; 3=pathologic`), and the first 21 column are the features. To see what each feature represents, take a look at the dataset info link above.

---

*import data*

In [4]:

ctg_data = pd.read_csv("ctg_data/CTG_clean.csv")


In [5]:

ctg_data.head()


,LB,AC,FM,UC,DL,DS,DP,ASTV,MSTV,ALTV,...,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,NSP
0,120,0.000,0.0,0.000,0.000,0.0,0.0,73,0.5,43,...,62,126,2,0,120,137,121,73,1,2
1,132,0.006,0.0,0.006,0.003,0.0,0.0,17,2.1,0,...,68,198,6,1,141,136,140,12,0,1
2,133,0.003,0.0,0.008,0.003,0.0,0.0,16,2.1,0,...,68,198,5,1,141,135,138,13,0,1
3,134,0.003,0.0,0.008,0.003,0.0,0.0,16,2.4,0,...,53,170,11,0,137,134,137,13,1,1
4,132,0.007,0.0,0.008,0.000,0.0,0.0,16,2.4,0,...,53,170,9,0,137,136,138,11,1,1


---

*get features names*

In [15]:

ctg_features = []
for col in ctg_data.columns:
    ctg_features.append(col)

ctg_features.pop() # remove the label (last) column

print(ctg_features)
print("\nNumber of features:", len(ctg_features) )


['LB', 'AC', 'FM', 'UC', 'DL', 'DS', 'DP', 'ASTV', 'MSTV', 'ALTV', 'MLTV', 'Width', 'Min', 'Max', 'Nmax', 'Nzeros', 'Mode', 'Mean', 'Median', 'Variance', 'Tendency']

Number of features: 21


---

*build feature and label dataframe/series*

In [21]:

label = "NSP" # target

X_ctg = ctg_data[ctg_features]
y_ctg = ctg_data[label]


In [19]:

X_ctg.head()


,LB,AC,FM,UC,DL,DS,DP,ASTV,MSTV,ALTV,...,Width,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency
0,120,0.000,0.0,0.000,0.000,0.0,0.0,73,0.5,43,...,64,62,126,2,0,120,137,121,73,1
1,132,0.006,0.0,0.006,0.003,0.0,0.0,17,2.1,0,...,130,68,198,6,1,141,136,140,12,0
2,133,0.003,0.0,0.008,0.003,0.0,0.0,16,2.1,0,...,130,68,198,5,1,141,135,138,13,0
3,134,0.003,0.0,0.008,0.003,0.0,0.0,16,2.4,0,...,117,53,170,11,0,137,134,137,13,1
4,132,0.007,0.0,0.008,0.000,0.0,0.0,16,2.4,0,...,117,53,170,9,0,137,136,138,11,1


In [20]:

y_ctg.head()


0    2
1    1
2    1
3    1
4    1
Name: NSP, dtype: int64

#### b.

  * Use `sklearn.preprocessing.scale` function to normalize the features.

In [30]:
X_ctg_scaled = scale(X_ctg)

X_ctg_scaled

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  """Entry point for launching an IPython kernel.


array([[-1.35222005, -0.8223883 , -0.20320955, ..., -1.18164215,
         1.87056871,  1.11298001],
       [-0.1325256 ,  0.73013282, -0.20320955, ...,  0.13203796,
        -0.23499819, -0.52452553],
       [-0.03088439, -0.04612774, -0.20320955, ..., -0.00624416,
        -0.2004807 , -0.52452553],
       ...,
       [ 0.68060404, -0.56363478, -0.20320955, ...,  0.96173066,
        -0.51113811,  1.11298001],
       [ 0.68060404, -0.56363478, -0.20320955, ...,  0.8925896 ,
        -0.51113811,  1.11298001],
       [ 0.88388645, -0.30488126, -0.16034157, ...,  0.47774325,
        -0.61469058, -0.52452553]])

#### c.

  * Design an ANN with one hidden layer with 30 neurons to predict the fetal’s health status. For your ANN, Use `random_state=1, learning_rate_init = 0.02, solver='adam', alpha=1, verbose=True, activation='logistic'`. Use 10-fold Cross-Validation to evaluate your model. Make sure to add “`verbose=True`” to see the training process. Then, Test your ANN on testing set, and calculate and report the accuracy.

d.
  * Fix the random state for reproducibility:
  
  `seed = 1, np.random.seed(seed)`.
  
  Now, use `GridSearchCV` to find the best number of neurons for your 1-hidden layer network.
  
  Search in the `range` of (5-250) with the step size of 5 for the number of neurons (i.e. 5 neurons, 10 neurons, 15 neurons, ... , 245 neurons, 250 neurons).
  
  As for other parameters, use the same arguments as part (c) for your network.
  
  What is the best accuracy, and best number of neurons?

## Problem 2:

### Face Recognition Using SVM

---